<a href="https://colab.research.google.com/github/jakubglinka/google.colab/blob/master/word2vec_nkjp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CBOW word2vec model on National Korpus of Polish Language

## Outline:
 
 - prepare data
 - train embeddings
 - train sentiment classifier

Remarks: use Tensorflow 2.0:

https://github.com/tensorflow/docs/blob/master/site/en/r2/guide/effective_tf2.md

### TODO's:
  - nkjp class with following functionalities:
    - download
    - summary
    - split into train, valid and test (using whole texts I guess) stratified by type of text
    - generate random sample of n sentences (segment on the fly) from traning/test/validation part
    - train segmentor
    - prepare tokenisation
  - sentencepeice tokenisation

### Install dependencies:

In [0]:
!pip install untangle tf-nightly-2.0-preview sentencepiece nltk wget tqdm

# Load corpus

http://nkjp.pl/settings/papers/NKJP_ksiazka.pdf


In [0]:
import subprocess
import tqdm
import os
import wget
import tarfile
import untangle as unt
from typing import List, Union, Tuple
import tqdm
import pandas as pd

class tqdm_wget_pbar:
  def __enter__(self):
    self.elapsed = None
    self.tqdm = None
    return self

  def update(self, elapsed, total, done):
    if self.tqdm is None:
      self.tqdm = tqdm.tqdm(total=round(total / 1e6), unit="MB", unit_scale=True)
      self.tqdm.update(round(elapsed / 1e6, 2))
      self.elapsed = elapsed
    else:
      self.tqdm.update(round((elapsed - self.elapsed) / 1e6, 5))
      self.elapsed = elapsed

  def __exit__(self, b, c, d):
    self.tqdm.close()

class NKJP:
 
  def __init__(self, dir: str, url: str = None):
    self.url = "http://clip.ipipan.waw.pl/NationalCorpusOfPolish?action=AttachFile&do=get&target=NKJP-PodkorpusMilionowy-1.2.tar.gz"
    self.dir = dir
  
  def is_downloaded(self) -> bool:  
    return os.path.exists(self.dir)
  
  def download(self):
        
    if self.is_downloaded() == False:
      
      os.mkdir(self.dir)
      with tqdm_wget_pbar() as pbar:
        _pbar = lambda elapsed, total, done: pbar.update(elapsed, total, done) 
        wget.download(url=self.url, 
                      out= self.dir + "/nkjp.tar.gz",
                      bar=_pbar)      
    else:
      print("nkjp already downloaded, reusing...")
      
    return self

  def is_extracted(self):
    nkjp_folders = [f for f in os.listdir(self.dir) if f != 'nkjp.tar.gz']
    return len(nkjp_folders) > 0
  
  def extract(self):
    
    if self.is_extracted() == False:
    
      with tarfile.open(self.dir + "/nkjp.tar.gz") as tar:
        tar.extractall(self.dir)
      
    else:
      print("nkjp already extracted, reusing...")
      
    return self
  
  def _get_text_stats(dir: str) -> List[Tuple[str, Union[str, int]]]:

    if os.path.isdir(dir):
      xml_header = unt.parse(dir + "/header.xml")
      text_types = xml_header.teiHeader.profileDesc.textClass.catRef
      text_types = [(xx['scheme'],xx['target']) for xx in text_types]

      num_words = int(xml_header.teiHeader.fileDesc.extent.num['value'])

      stats = text_types + [("words", num_words)]
      stats = dict(stats)


    else:
      stats = {}


    return [dir, [stats.get("words", None), 
                  stats.get("#taxonomy-NKJP-type", None), 
                  stats.get("#taxonomy-NKJP-channel", None)]]
                           

  def get_text_stats(self) -> pd.DataFrame:

    nkjp_folders = [f for f in os.listdir(self.dir) if f not in ['nkjp.tar.gz']]
    xx = [get_text_stats("./nkjp/" + folder) for folder in nkjp_folders]
    df = pd.DataFrame.from_items(xx, orient="index", columns=['words', 'type', 'channel'])

    return df
      
 

#### NKJP structure:

Typ tekstu:
1. Literatura piękna:
    -  proza,
    - poezja,
    - dramat, 
2. literatura faktu,
3. publicystyka i krótkie wiadomości prasowe,
4. typ naukowo-dydaktyczny,
5. typ informacyjno-poradnikowy,
6. książka niebeletrystyczna niesklasyfikowana,
7. inne teksty pisane
     - typ urzędowo-kancelaryjny,
     - teksty perswazyjne (ogłoszenia, reklamy, propaganda polityczna),
     - krótkie teksty instruktażowe ,
8. listy,
9. Internet
    - interaktywne strony WWW (fora, chaty, listy dyskusyjne itp.),
    - statyczne strony WWW,
10. teksty mówione konwersacyjne,
11. teksty mówione medialne,
12. teksty quasi-mówione.

In [262]:
# !rm -rf nkjp* NKJP*
# !ls

nkjp = NKJP("./nkjp/")
nkjp = nkjp.download()
nkjp = nkjp.extract()
df = nkjp.get_text_stats()
df.describe(include="all")

nkjp already downloaded, reusing...
nkjp already extracted, reusing...


,words,type,channel
count,3889.000000,3889,3889
unique,NaN,14,8
top,NaN,#typ_publ,#kanal_prasa_dziennik
freq,NaN,1986,1744
mean,256.421702,NaN,NaN
std,1126.625693,NaN,NaN
min,0.000000,NaN,NaN
25%,47.000000,NaN,NaN
50%,55.000000,NaN,NaN
75%,197.000000,NaN,NaN


# Sentence generator
## Segmentation reader:


In [265]:
import untangle as unt
from untangle import Element
import itertools
from typing import Tuple, Dict, List
import re
from itertools import groupby
from functools import reduce


def try_or_none(el):
    try:
      res = [seg.seg for seg in el.s]
    except:
        res = None

    return res 
  
def _get_token(txt: str) -> Tuple[str, int, int]:
  pattern = re.compile('\((.*)\)')
  res = re.search(pattern, txt)[0].strip("()").split(",")
  return tuple(res)

def _get_tokens(elts: List[Element]) -> Tuple[str, List[Tuple[int, int]]]:
  tokens = [_get_token(el['corresp']) for el in elts]
  id = tokens[0][0]
  tokens = list(map(lambda x: (int(x[1]), int(x[2])), tokens))
  return id, [tokens]


def _merge(x, y):
  return x[0], x[1] + y[1]
  

def get_tokens(dir: str) -> Dict[str, List[Tuple[int, int]]]:

  xml_segs = unt.parse(dir + "/ann_segmentation.xml")
  xml_segs = xml_segs.teiCorpus.TEI.text.body.p
  xml_segs = list(itertools.chain(*[try_or_none(xx) for xx in xml_segs]))
  xml_segs = [_get_tokens(xx) for xx in xml_segs]
  xml_segs = [reduce(_merge, groups) for (key, groups) in groupby(xml_segs, lambda x: x[0])]
  return dict(xml_segs)

get_tokens("./nkjp/TochmanWsciekly/")["txt_1.2-ab"]







[[(0, 4), (5, 1), (7, 4), (11, 1)],
 [(13, 1), (15, 2), (18, 6), (25, 9), (34, 1)]]

# Text pre-processing:

http://www.aclweb.org/anthology/P16-1162

  - sentencepiece
  - to train senence piece model we need to prepare ...

In [162]:
# generate text file with tokenized sentences 



xx = list(get_texts("./nkjp/TochmanWsciekly/").values())
with open("texts.txt", "w") as f:
  for l in xx:
    f.write("<s>" + l + "</s>" + "\n")
  
!cat texts.txt

<s>Tych, którzy krzyczą o homofobii, prawda w oczy kole.</s>
<s>Kole i mnie. I ja jestem zaburzony.</s>
<s>Niektórzy mówią, że od homoseksualizmu do pedofilii droga niedaleka. Nie wydaje mi się. Ja na chłopców nie patrzę. Mnie chłopcy nie interesują.</s>
<s>Mnie się podobają dojrzali mężczyźni, śniadzi, wysocy, mocni. Ich szukam, ich potrzebuję.</s>
<s>I to jest mała część prawdy, na początek.</s>
<s>Odsunął mnie od siebie.</s>
<s>Rok dla chłopaka to era.</s>
<s>Po roku nie było go już w naszej parafii. Pojechał do Afryki pracować z chorymi na AIDS.</s>
<s>Byłem zupełnie sam z moim pragnieniem męskiego dotyku.</s>
<s>Ale nie pamiętam pierwszego seksu. To znaczy nie wiem, który był pierwszy. Chyba kiedy byłem już w seminarium.</s>
<s>Ważny ślad, który zaprowadził nas donikąd. Ślepa uliczka - nikt w MZA Janka nie poznaje.</s>
<s>Jan bardzo chce wiedzieć, kim jest, mieć na krzyżu nazwisko, kiedy przyjdzie jego czas. Ale nie boi się śmierci. Sądzi, że śmierci nie ma. Ciało, które ginie, je

# Generate Skip-gram batches

https://arxiv.org/abs/1310.4546

### Texts

In [175]:
# !ls ./nkjp/GazetaPomorska/

# !head ./nkjp/TochmanWsciekly/text.xml -n 50
# !head ./nkjp/TochmanWsciekly/ann_segmentation.xml -n 50

import untangle as unt
from typing import Dict
import itertools

def get_texts(dir: str) -> Dict[str, str]:
  
  xml_texts = unt.parse(dir + "/text.xml")
  xml_texts = [xxx for xxx in xml_texts.teiCorpus.TEI.text.body.div]

  def try_or_none(el):
    try:
      res = [(txt['xml:id'], txt.cdata) for txt in el.ab]
    except:
      try:
        res = [(txt['xml:id'], txt.cdata) for txt in el.u]
      except:
        try:
          res = [(txt['xml:id'], txt.cdata) for txt in el.p]
        except:
          res = None

    return res 

  xml_texts = list(itertools.chain(*[try_or_none(xx) for xx in xml_texts]))
  return dict(xml_texts)

get_texts("./nkjp/TochmanWsciekly/")["txt_1.2-ab"]

'Kole i mnie. I ja jestem zaburzony.'

3905

sample_data


151MB [00:11, 13.3MB/s]                         


[]

In [0]:
nkjp is None

False

In [0]:
import tarfile
tar = tarfile.open("./nkjp/nkjp.tar.gz")
tar
tar.extractall("./nkjp/")
tar.close()

In [0]:
!ls ./nkjp/ | head -n 5

010-2-000000001
030-2-000000001
030-2-000000002
030-2-000000003
030-2-000000004


In [108]:
!echo "To jest mój tekst w języku Polskim ." > text.txt
!echo "Zdanie drugie . To jest moje drugie zdanie ." >> text.txt

!echo "To jest mój tekst w języku Polskim ." > text2.txt


!head text.txt

To jest mój tekst w języku Polskim .
Zdanie drugie . To jest moje drugie zdanie .


In [109]:
!pwd

/content


In [166]:
import sentencepiece as spm
spm.SentencePieceTrainer.Train("--input=/content/texts.txt --model_prefix=m --vocab_size=100 --model_type=unigram")
sp = spm.SentencePieceProcessor()
sp.Load("/content/m.model")


ids = sp.EncodeAsIds("""To jest mój tekst w języku Polskim , słowo nieznane . \n To jest drugie zdanie .""")
print(ids)
sp.DecodeIds(ids)

# sp.EncodeAsPieces("This is a test")

[3, 55, 4, 56, 3, 10, 65, 19, 3, 16, 8, 14, 15, 16, 3, 99, 3, 19, 26, 11, 9, 14, 13, 3, 87, 4, 42, 15, 36, 10, 3, 12, 31, 30, 4, 99, 4, 28, 11, 22, 5, 22, 8, 3, 6, 3, 55, 4, 56, 3, 18, 24, 13, 25, 21, 3, 11, 18, 5, 17, 3, 6]


'To jest mój tekst w języku Polskim , słowo nieznane . To jest drugie zdanie .'

In [150]:
sp.id_to_piece(3)

'▁'

In [0]:
import tensorflow as tf
import untangle as unt

In [0]:
# download and extract korpus from ipipan website

!wget "http://clip.ipipan.waw.pl/NationalCorpusOfPolish?action=AttachFile&do=get&target=NKJP-PodkorpusMilionowy-1.2.tar.gz" -O nkjp.tar.gz
!mkdir nkjp
!tar -C ./nkjp -zxf nkjp.tar.gz -o
!ls

In [0]:
!ls ./nkjp/ | head -n 2
!cat ./nkjp/030-2-000000001/text.xml

010-2-000000001
030-2-000000001
<?xml version="1.0" encoding="UTF-8"?>
<teiCorpus xmlns:xi="http://www.w3.org/2001/XInclude" xmlns="http://www.tei-c.org/ns/1.0">
 <xi:include href="NKJP_1M_header.xml"/>
 <TEI>
  <xi:include href="header.xml"/>
  <text xml:id="txt_text" xml:lang="pl">
   <body xml:id="txt_body">
    <div xml:id="txt_1-div" decls="#h_1-bibl">
     <ab n="p882in890of:PWN:030-2-000000001" xml:id="txt_1.1-ab">Łzy padały na cremoński lakier. Szkoda, nie wolno było niszczyć przedmiotu, na który ojciec, biedaczysko, wydał całą schedę po Luizie... Otarła je lewą, umęczoną ręką.</ab>
     <ab n="p883in891of:PWN:030-2-000000001" xml:id="txt_1.2-ab">Wszedł Adam. Zawiało wodą kwiatową Maréchal Niel. Starannie domykał drzwi za sobą.</ab>
     <ab n="p884in892of:PWN:030-2-000000001" xml:id="txt_1.3-ab">Trwała dalej w bezruchu.</ab>
    </div>
    <div xml:id="txt_2-div" decls="#h_2-bibl">
     <ab n="p45in53of:PWN:030-2-000000001" xml:id="txt_2.1-ab">Pan dyrektor był nieobecny, a Róż

In [0]:
xx = unt.parse("./nkjp/030-2-000000001/text.xml")
[xxx.cdata for xxx in xx.teiCorpus.TEI.text.body.div[11].ab]

['Róża opuściła ręce. Siadła - nogi drżały. Rzuciła smyczek...',
 'Księżycowa orkiestra pod batutą Brahmsa grała dalej. Tylko na miejscu skrzypiec wystąpiła cisza - czarna jak zaskórna woda. Jeszcze tu i ówdzie błyskał refleks sola... cisza przecież czyniła się coraz głębsza, szersza i pochłaniała resztkę wibracji. Z wolna milkły także widmowe instrumenty. Po jednemu wsiąkały w próżnię... Chaos bezdźwięczny pokrył wreszcie harmonię.']

In [0]:
import tensorflow as tf

In [0]:
tf.__version__

'2.0.0-dev20190217'

In [0]:
x = tf.random.normal([2, 2])

@tf.function
def square(x: tf.Tensor) -> tf.Tensor:
  return(tf.square(x))

print(x)
print(square(x))

W0217 10:25:40.323168 140638079080320 tf_logging.py:161] Entity <function square at 0x7fe8b0f67f28> could not be transformed and will be staged without change. Error details can be found in the logs when running with the env variable AUTOGRAPH_VERBOSITY=5. Please report this to the AutoGraph team. Cause: Unexpected error transforming <function square at 0x7fe8b0f67f28>. If you believe this is due to a bug, please set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output when filing the bug report. Caused by: name 'tf' is not defined


tf.Tensor(
[[-0.15044403  1.6586291 ]
 [ 1.1326104  -1.2539116 ]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[0.02263341 2.7510505 ]
 [1.2828064  1.5722944 ]], shape=(2, 2), dtype=float32)


In [0]:
??tf.function